# Rushi Patel

In [14]:
import re
import pandas as pd
from pathlib import Path

# ✅ Update this if your file name is different
INPUT_PATH = "Canada.xlsx - Regions by Citizenship.csv"


df = load_un_migration_csv(INPUT_PATH)
df.head()


,Type,Coverage,AreaName,RegName,1980,1981,1982,1983,1984,1985,...,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012
0,Immigrants,Citizens,Northern America,Northern America,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,1,1
1,Immigrants,Foreigners,Africa,Eastern Africa,1471,1641,1426,1094,1187,1134,...,5808,7726,7083,6750,6669,6705,6922,8591,8956,8846
2,Immigrants,Foreigners,Africa,Middle Africa,33,27,44,32,42,30,...,1929,2452,2802,2834,2951,3034,3255,3369,3146,4602
3,Immigrants,Foreigners,Africa,Northern Africa,1100,1268,1348,936,842,840,...,11355,12478,11323,12705,11750,13456,16320,19271,15712,15453
4,Immigrants,Foreigners,Africa,Southern Africa,1041,1126,791,387,297,327,...,1258,1203,1012,1148,1230,1184,1235,1306,1049,1348


In [12]:
# Identify all year columns automatically (1980, 1981, ...)
year_cols = [c for c in df.columns if re.fullmatch(r"\d{4}", str(c))]

# Replace '..' with 0 and convert all year columns to integers
for c in year_cols:
    df[c] = pd.to_numeric(df[c].replace("..", 0), errors="coerce").fillna(0).astype(int)

# (Optional) Filter to immigrants only (adjust as your assignment requires)
if "Coverage" in df.columns:
    # Keeps immigrants, including 'Foreigners' and 'Citizens' rows (edit as needed)
    df = df[df["Type"] == "Immigrants"].copy()

df[["Type"] + (["Coverage"] if "Coverage" in df.columns else []) + year_cols[:5]].head()


,Type,Coverage,1980,1981,1982,1983,1984
0,Immigrants,Citizens,0,0,0,0,0
1,Immigrants,Foreigners,1471,1641,1426,1094,1187
2,Immigrants,Foreigners,33,27,44,32,42
3,Immigrants,Foreigners,1100,1268,1348,936,842
4,Immigrants,Foreigners,1041,1126,791,387,297


In [4]:
# Build the tidy output frame in the required order.
out = pd.DataFrame()

# If the dataset includes OdName (country of origin), use it.
# If not (region-only export), fall back to RegName so you still get a usable 'Country' column.
if "OdName" in df.columns:
    out["Country"] = df["OdName"]
elif "RegName" in df.columns:
    out["Country"] = df["RegName"]
else:
    out["Country"] = df.index.astype(str)

# These exist in some exports (yours includes AreaName/RegName; OdName/DevName may vary by sheet)
out["Continent"] = df["AreaName"] if "AreaName" in df.columns else ""
out["Region"]    = df["RegName"] if "RegName" in df.columns else ""
out["DevName"]   = df["DevName"] if "DevName" in df.columns else ""

# Add all year columns
for c in year_cols:
    out[c] = df[c].values

# Add Total
out["Total"] = out[year_cols].sum(axis=1)

out.head()


,Country,Continent,Region,DevName,1980,1981,1982,1983,1984,1985,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,Total
0,Northern America,Northern America,Northern America,,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,18
1,Eastern Africa,Africa,Eastern Africa,,1471,1641,1426,1094,1187,1134,...,7726,7083,6750,6669,6705,6922,8591,8956,8846,157941
2,Middle Africa,Africa,Middle Africa,,33,27,44,32,42,30,...,2452,2802,2834,2951,3034,3255,3369,3146,4602,43623
3,Northern Africa,Africa,Northern Africa,,1100,1268,1348,936,842,840,...,12478,11323,12705,11750,13456,16320,19271,15712,15453,241469
4,Southern Africa,Africa,Southern Africa,,1041,1126,791,387,297,327,...,1203,1012,1148,1230,1184,1235,1306,1049,1348,40135
